# 01  Create Azure Environment

## Azure 

This notebook contains the script to create the necessary Azure environment to run the provided samples. The notebook uses [PowerShell](https://learn.microsoft.com/de-de/powershell/scripting/install/installing-powershell?view=powershell-7.3) and [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli) with the BICEP command group to deploy all necessary Azure resources. Both tools are available on Windows, macOS and Linux environments.

## Step 1:   Login to Azure; get, set subscription

In [1]:
# Check if you are already logged in
$loggedIn = az account show --query "name" -o tsv

if ($loggedIn -ne $null) {
    Write-Host "Already logged in as $loggedIn"
} else {
    Write-Host "Logging in..."
    az login
}
# Retrieve default subscription id
$subscriptionId = (
    (
        az account list -o json `
            --query "[?isDefault]"
    ) | ConvertFrom-Json
).id

# Set Subscription
az account set --subscription $subscriptionId

Write-Host "Subscription set to $subscriptionId"

Already logged in as CTE - Courseware Development 14 [COGS]


Error: Microsoft.DotNet.Interactive.NoSuitableKernelException: No kernel found for Microsoft.DotNet.Interactive.Commands.RequestHoverText with target kernel 'pwsh'.
   at Microsoft.DotNet.Interactive.Commands.KernelCommand.InvokeAsync(KernelInvocationContext context) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Commands\KernelCommand.cs:line 184
   at Microsoft.DotNet.Interactive.Kernel.HandleAsync(KernelCommand command, KernelInvocationContext context) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Kernel.cs:line 323
   at Microsoft.DotNet.Interactive.KernelCommandPipeline.<BuildPipeline>b__6_0(KernelCommand command, KernelInvocationContext context, KernelPipelineContinuation _) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\KernelCommandPipeline.cs:line 60
   at Microsoft.DotNet.Interactive.KernelCommandPipeline.SendAsync(KernelCommand command, KernelInvocationContext context) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\KernelCommandPipeline.cs:line 41

Error: Command cancelled.

## Step 2:   Define project unifier


In [2]:
$unifier = Get-Random -Minimum 100 -Maximum 999

Write-Host "Unifier set to: $unifier"

Unifier set to: 850


## Step 3:   Create Resource Group

In [2]:
$resourceGroup = "LIL-AOAI"
$location = "canadaeast"

az group create `
    --name $resourceGroup `
    --location $location 

Write-Host "Resource group $resourceGroup created in $location..."

{
  "id": "/subscriptions/56d848f2-9cd5-4581-b998-80dc5df9bae9/resourceGroups/LIL-AOAI",
  "location": "canadaeast",
  "managedBy": null,
  "name": "LIL-AOAI",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}
Resource group LIL-AOAI created in canadaeast...


## Step 4:   Deploy Azure Resources

### Step 4.1: Prepare deployment

In [4]:
$deploymentParametersFile = "./azuredeploy.parameters.json"

$deploymentTemplate = Get-Content (Join-Path -Path (Get-Location) -ChildPath $deploymentParametersFile)
$deploymentTemplate = $deploymentTemplate -replace "unifier", $unifier

$deploymentTemplate | Set-Content (Join-Path -Path (Get-Location) -ChildPath $deploymentParametersFile)

Write-Host "Unifier $unifier added to service names..."

Unifier 850 added to service names...


### Step 4.2: Deployment

In [3]:
$deploymentName = "LIL-AOAI"
$deploymentTemplateFile = "azuredeploy.bicep"
$deploymentParametersFile = "azuredeploy.parameters.json"

$deployment = ( 
    az deployment group create `
        --name $deploymentName `
        --resource-group $resourceGroup `
        --template-file $deploymentTemplateFile `
        --parameters @$deploymentParametersFile `
) | ConvertFrom-Json

Write-Host "Deployment $deploymentName created..."

Deployment LIL-AOAI created...


## Step 5: Create application.env file


In [4]:
$aoaiApiKey = $deployment.properties.outputs.aoai_apikey.value
$aoaiEndpoint = $deployment.properties.outputs.aoai_endpoint.value
$embeddingDeploymentName = $deployment.properties.outputs.embedding_deploymentname.value
$chatCompletionDeploymentName = $deployment.properties.outputs.chatcompletion_deploymentname.value
$cognitiveSearchApiKey = $deployment.properties.outputs.cognitivesearch_apikey.value
$cognitiveSearchEndpoint = $deployment.properties.outputs.cognitivesearch_endpoint.value

# Create configuration file for .NET demo applications
$configurationFile = Join-Path -Path (Get-Location) -ChildPath "application.env"
Add-Content -Path $configurationFile -value "AOAI_APIKEY = $aoaiApiKey"
Add-Content -Path $configurationFile -value "AOAI_ENDPOINT = $aoaiEndpoint"
Add-Content -Path $configurationFile -value "CHATCOMPLETION_DEPLOYMENTNAME = $chatCompletionDeploymentName"
Add-Content -Path $configurationFile -value "EMBEDDING_DEPLOYMENTNAME = $embeddingDeploymentName"
Add-Content -Path $configurationFile -value "COGNITIVESEARCH_ENDPOINT = $cognitiveSearchEndpoint"
Add-Content -Path $configurationFile -value "COGNITIVESEARCH_APIKEY = $cognitiveSearchApiKey"

Write-Host "Configuration file created at: $configurationFile..."

# Copy configuration file to chapter 02 folder
copy $configurationFile "../02/.env"

Write-Host "Configuration file copied to chapter 02 folder..."

Configuration file created at: C:\LiL_AOAI\01\application.env...
Configuration file copied to chapter 02 folder...
